In [117]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate

In [118]:
# Data ingestion
path = "data/EduTrack_FAQ_assignment.pdf"
pdf_docs = PyPDFLoader(path).load()
pdf_docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-06-05T12:53:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-06-05T12:53:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'data/EduTrack_FAQ_assignment.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='EduTrack – Frequently Asked Questions\nQ1: What is EduTrack used for?\nA1: EduTrack helps educational institutions monitor student engagement, analyze learning\nbehavior, and proactively support at-risk learners through data-driven insights.\nQ2: Which platforms does EduTrack integrate with?\nA2: EduTrack integrates seamlessly with LMS platforms such as Moodle, Canvas,\nBlackboard, Google Classroom, and can be extended to custom LMS solutions via API.\nQ3: What types of student data does EduTrack analyze?\nA3: It analyzes logins, session duration, assignment submission patterns, q

In [119]:
# Chunking
chunker = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 10,
)

chunked_docs = chunker.split_documents(pdf_docs)
chunked_docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-06-05T12:53:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-06-05T12:53:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'data/EduTrack_FAQ_assignment.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='EduTrack – Frequently Asked Questions\nQ1: What is EduTrack used for?\nA1: EduTrack helps educational institutions monitor student engagement, analyze learning\nbehavior, and proactively support at-risk learners through data-driven insights.\nQ2: Which platforms does EduTrack integrate with?\nA2: EduTrack integrates seamlessly with LMS platforms such as Moodle, Canvas,\nBlackboard, Google Classroom, and can be extended to custom LMS solutions via API.'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-0

In [120]:
# Vectorise and store
embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)
vector_db = Chroma.from_documents(chunked_docs, embedding)

In [121]:
res = vector_db.similarity_search("How does EduTrack predict dropout risks")
print("".join([doc.page_content for doc in res]))

Q3: What types of student data does EduTrack analyze?
A3: It analyzes logins, session duration, assignment submission patterns, quiz scores,
discussion participation, video viewing activity, and more.
Q4: How does EduTrack predict dropout risks?
A4: The platform uses machine learning algorithms that flag students at risk based on
inactivity, declining academic performance, missed deadlines, and low participation.
Q5: How does EduTrack benefit teachers?Q3: What types of student data does EduTrack analyze?
A3: It analyzes logins, session duration, assignment submission patterns, quiz scores,
discussion participation, video viewing activity, and more.
Q4: How does EduTrack predict dropout risks?
A4: The platform uses machine learning algorithms that flag students at risk based on
inactivity, declining academic performance, missed deadlines, and low participation.
Q5: How does EduTrack benefit teachers?A18: Yes. It can sync with most SIS platforms to fetch enrollment, demographics, and
aca

In [122]:
# Query the llm with context pulling
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_ollama.chat_models import ChatOllama

prompt = PromptTemplate.from_template('''
                                          You are a helpful assistant who only answers to any question based on the provided context. 
                                          
                                          <context>{context}</context> 
                                           
                                          Question: {input}

                                          NOTE: DO NOT INCLUDE RESULTS/PIECES OF RETIRVED CONTEXT DIRECTLY TO THE USER THEY MAY CONTAIN UNFORMATTED SYMBOLS. IF
                                          YOU REALLY WANT TO SHOW SOMETHING THEN FIRST CLEAN IT AND THEN SHOW.
                                          ''')
llm = ChatOllama(model="llama3.1:8b")
stuff_doc_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, stuff_doc_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x30256deb0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n                                          You are a helpful assistant who only answers to any question based on the provided context. \n                                          \n                                          <context>{context}</context> \n                   

In [123]:
results = retrieval_chain.invoke({
    "input": "How does EduTrack benefit teachers?"
})
results['answer']

"EduTrack benefits teachers by providing them with valuable insights and tools to support student success. The platform helps teachers identify at-risk students early on, allowing them to intervene and provide targeted support. This enables teachers to tailor their instruction to meet the needs of all students, improve academic performance, and ultimately reduce dropout risks.\n\nAdditionally, EduTrack's weekly alerts and visual cues help faculty stay informed about students who need attention, making it easier for them to manage their classrooms effectively.\n\nIn short, EduTrack supports teachers by providing data-driven insights, enabling early intervention, and facilitating targeted support to improve student outcomes."

In [124]:
results = retrieval_chain.invoke({
    "input": "How does EduTrack benefit students?"
})
results['answer']

"Unfortunately, the provided context does not directly address how EduTrack benefits students. However, based on other questions and answers, we can infer that EduTrack might help identify at-risk students (Q19 & A19) and provide early intervention opportunities.\n\nConsidering this indirect information, a possible answer to your question could be:\n\nEduTrack's ability to predict dropout risks (A4) and notify instructors about at-risk students (A19) may ultimately benefit students by providing them with timely support and resources to improve their academic performance."

In [125]:
results = retrieval_chain.invoke({
    "input": "Can I self host it?"
})
results['answer']

'Yes, you can self-host EduTrack. It offers both cloud-hosted SaaS and on-premise deployments for institutions with specific data residency needs.'

In [126]:
results = retrieval_chain.invoke({
    "input": "Does it have customizable reporting or dashboards?"
})

print(results['answer'])

Based on the provided context, there is no direct mention of customizable reporting or dashboards. However, it can be inferred that instructors have access to a dashboard with visual cues and tools to send personalized feedback or motivational nudges (Q5), and students can view their own learning progress on their dashboards (Q6).

But, one related information suggests that the application sends push notifications for approaching deadlines, grade changes etc. which can indirectly imply some form of customizable reporting or alert system based on user-defined settings or thresholds.

To provide a more accurate answer: There is no explicit mention of customizable reporting, but it's possible that some level of customization exists through defined risk thresholds (A19) and the ability to send personalized feedback or motivational nudges.
